In [ ]:
from sklearn import metrics

In [ ]:
#LSTM models
# input shape format is [samples, timesteps, features]
# base has [30, 1, 9]

In [ ]:
def format_to_lstm(df):
    X = np.array(df)
    return np.reshape(X, (X.shape[0], 1, X.shape[1]))

In [ ]:
# Transform the datasets for LSTM processing
lstm_X_test1_summaryL1Z2_std = format_to_lstm(X_test1_summaryL1Z2_std)
lstm_X_test1_summaryL1Z2_minMax = format_to_lstm(X_test1_summaryL1Z2_minMax)
lstm_X_train_base_std = format_to_lstm(X_train_base_std)
lstm_X_train_base_minMax = format_to_lstm(X_train_base_minMax)
lstm_X_train_all_std = format_to_lstm(X_train_all_std)
lstm_X_train_all_minMax = format_to_lstm(X_train_all_minMax)
lstm_X_test_all_std = format_to_lstm(X_test_all_std)
lstm_X_test_all_minMax = format_to_lstm(X_test_all_minMax)

In [ ]:
# Neural Nets imports
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adadelta, Adagrad, Adamax, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l1, l2, l1_l2

In [ ]:
from tensorflow.keras.utils import multi_gpu_model

In [ ]:
#Define LSTM model
#input shape for the LSTM is (number_steps x number_features)

def lstm_model_mse(layerSize, numSteps, numFeatures):
    
    model = Sequential()
    model.add(LSTM(layerSize, return_sequences=True, input_shape=(numSteps,numFeatures),
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),                   
                   dropout=0.2, recurrent_dropout=0.2))
    model.add(LSTM(layerSize, input_shape=(numSteps,numFeatures), 
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                   dropout=0.2, recurrent_dropout=0.2))

    model.add(Dense(layerSize, kernel_initializer='normal',
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                    activation=custom_activation))
    #model.add(BatchNormalization())
    
    model.add(Dense(1, kernel_initializer='normal', 
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                    activation='linear'))

    model.compile(Adam(lr=0.001),
              loss='mse',
              metrics=['mse'])
    return model


In [ ]:
verboseLevel=0
validationSplit=0.2
batchSize=30
epochs=1000

In [ ]:
def createReduceRLObject():
    reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.5,
                              patience=2,
                              verbose=verboseLevel,
                              mode='min',
                              min_lr=0.001)
    return reduce_lr

In [ ]:
layerSize = 64
numSteps = 1
numFeatures = 9

In [ ]:
#### END ####

In [ ]:
# The below is how to create multivariate time series with multiple steps from a single step multivariate series
def format_to_lstm_nsteps(df_X, df_Y, n_steps=1):
    X, y = list(), list()
    df_X_a = df_X.to_numpy()
    for i in range(len(df_X)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(df_X):
            break
        # gather input and output parts of the pattern
        seq_x = df_X_a[i:end_ix, :]
        seq_y = df_Y[end_ix-1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
X_train_lstm3, y_train_lstm3 = format_to_lstm_nsteps(X_train_base_std, y_train_scaled_std_base,3)

In [ ]:
lstm3_std_base_mse_model = lstm_model_mse(layerSize=layerSize, numSteps=3, numFeatures=numFeatures)

In [ ]:
checkpoint_lstm3_std_base_mse = ModelCheckpoint("lstm3_model_std_base_mse_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')
reduce_lr_std_base_mse3 = createReduceRLObject()
callbacks_list_lstm3_std_base_mse = [checkpoint_lstm3_std_base_mse, reduce_lr_std_base_mse3]

In [ ]:
%%time
history_LSTM3_std_base_mse = lstm3_std_base_mse_model.fit(X_train_lstm3, y_train_lstm3,
                                batch_size=5, 
                                validation_split=validationSplit, 
                                epochs=epochs, verbose=verboseLevel,
                                callbacks=callbacks_list_lstm3_std_base_mse)

In [ ]:
X_test_lstm3, y_test_lstm3 = format_to_lstm_nsteps(X_test1_summaryL1Z2_std, y_test_scaled_std_base, 3)

In [ ]:
#Reload the best saved model
lstm3_std_base_mse_model_new = load_model('lstm3_model_std_base_mse_L2.h5')
#Predict
y_pred_LSTM_base_std_L1Z2_mse = lstm3_std_base_mse_model_new.predict(X_test_lstm3)
pred = targetStdScalerBase.inverse_transform(y_pred_LSTM_base_std_L1Z2_mse)
y_target = targetStdScalerBase.inverse_transform(y_test_lstm3)

In [ ]:
minMseAWS_base = metrics.mean_squared_error(y_target, pred)
minRmseAWS_base = np.sqrt(minMseAWS_base)
minMaeAWS_base = metrics.mean_absolute_error(y_target, pred)
minR2AWS_base = metrics.r2_score(y_target, pred)
modelNameAWS_base = "LSTM3_STD_BASE_MSE"

In [ ]:
delta = pd.DataFrame()
delta = (1-(pd.DataFrame(pred)  / pd.DataFrame(y_target)))*100
delta = delta.rename(columns={delta.columns[0]: "ThroughputDeltaDeviationPercentage"})

In [ ]:
mpd_base = delta.median()[0]

In [ ]:
def mape(test, pred):
    x = 0
    for i in range(len(test)):
        x+=np.abs((test[test.columns[0]][i] - pred[i])/test[test.columns[0]][i])
    return (x/len(test)*100)

In [ ]:
mape_base = np.mean(np.abs((y_target - pred) / y_target)) * 100

In [ ]:
minMaeAWS_base, minR2AWS_base, modelNameAWS_base, minMseAWS_base, minRmseAWS_base